# HOMEWORK 3 
# CS584 MACHINE LEARNING 
# JULEN FERRO BAÑALES
This Jupyter Notebook will include the coding of the Homework 3 related to the subject CS584 Machine Learning, as well as some comments made.

In [575]:
import numpy as np
import pandas as pd
import matplotlib as plt
import csv
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler



Now, some basic auxiliary functions will be defined.

In [576]:

def cleaning(df):
    df = df.drop(['rightdown'], axis=1, level= 1)
    df = df.drop(['topleft'], axis=1, level= 1)
    return df

The first thing that must be done, is to read the CSV file with the location data that has been obtained after processing the mice images.

In [577]:
 
# reading the CSV file
df = pd.read_csv(r"C:\Users\ferro\Desktop\Disco Duro (17-08-2022) PRE-CHICAGO\DRIVE\University ( 28-11-2020 )\Master\Chicago IIT\Julen Subjects CHICAGO IIT\S1\CS 584 Machine Learning\Assignments\HW_3\A20512110_CollectedData_annotation.csv", header = [0, 1, 2, 3])
df = pd.DataFrame(df) 

size = os.path.getsize(r"C:\Users\ferro\Desktop\Disco Duro (17-08-2022) PRE-CHICAGO\DRIVE\University ( 28-11-2020 )\Master\Chicago IIT\Julen Subjects CHICAGO IIT\S1\CS 584 Machine Learning\Assignments\HW_3\A20512110_CollectedData_annotation.csv")

# displaying the contents of the CSV file
df.head()


scorer Unnamed: 1_level_0  \
    individuals Unnamed: 1_level_1   
      bodyparts Unnamed: 1_level_2   
         coords Unnamed: 1_level_3   
0  labeled-data               24.0   
1  labeled-data               24.0   
2  labeled-data               24.0   
3  labeled-data               24.0   
4  labeled-data               24.0   

                                  Unnamed: 2_level_0  annotation              \
                                  Unnamed: 2_level_1      mouse1               
                                  Unnamed: 2_level_2     topleft               
                                  Unnamed: 2_level_3           x           y   
0  A_male_in_a_new_cage_face_view_3_2022-08-10_15...  273.048052   12.903231   
1  A_male_in_a_new_cage_face_view_3_2022-08-10_15...  286.152150  140.449787   
2  A_male_in_a_new_cage_face_view_3_2022-08-10_15...  238.977396  154.427492   
3  A_male_in_a_new_cage_face_view_3_2022-08-10_15...  230.241331  166.657984   
4  A_male_in_a_new_cage_face_view_3_2022-08-10_15...  233.735757  173.646836   

                                                               ...           \
                                                               ...   mouse2   
    rightdown                    nose                 leftear  ... rightear   
            x           y           x           y           x  ...        x   
0  544.739689  331.769622  517.657886   21.639296  504.553788  ...      NaN   
1  516.784280  315.171098  515.910673  175.394049  488.828870  ...      NaN   
2  394.479362  225.189623  247.713462  212.085524  267.806412  ...      NaN   
3  310.613133  239.167328  287.899363  192.866180  273.048052  ...      NaN   
4  299.256248  261.007491  240.724609  187.624541  251.207888  ...      NaN   

                                                         
                                                         
      leftHip     rightHip     tailBase     tailEnd      
    y       x   y        x   y        x   y       x   y  
0 NaN     NaN NaN      NaN NaN      NaN NaN     NaN NaN  
1 NaN     NaN NaN      NaN NaN      NaN NaN     NaN NaN  
2 NaN     NaN NaN      NaN NaN      NaN NaN     NaN NaN  
3 NaN     NaN NaN      NaN NaN      NaN NaN     NaN NaN  
4 NaN     NaN NaN      NaN NaN      NaN NaN     NaN NaN  

[5 rows x 39 columns]

 Now, the dataset will be processed in order to get the dataset structure needed to feed the Neural Network. 

In [578]:
rows = len(df) #getting the number of data examples that we have
df = df.iloc[: , 3:-1]
mouse1 = df.xs('mouse1', level=1, axis=1).dropna()
mouse2 = df.xs('mouse2', level=1, axis=1).dropna()

# we add as rows the mouse2 samples' data taken in pictures where more than one mice were found
df = pd.concat([mouse1, mouse2], axis=0)

# we create one dataset per axis with data related to its own axis
df_x = df.xs('x', level=2, axis=1)
df_y = df.xs('y', level=2, axis=1)

# we calculate the average location of the mice in both axis
x_cg = (df_x.xs('topleft', level=1, axis=1) + df_x.xs('rightdown', level=1, axis=1)) / 2
y_cg = (df_y.xs('topleft', level=1, axis=1) + df_y.xs('rightdown', level=1, axis=1)) / 2

# renaming the columns of the dataframes
x_cg.rename(columns = {'annotation':'x_location'}, inplace = True)
y_cg.rename(columns = {'annotation':'y_location'}, inplace = True)

# we get rid of the data used for calculating the average location of the mice in both axis
df = cleaning(df)
df_x = cleaning(df_x)
df_y = cleaning(df_y)

# we add to df the calculated average location of the mice in both axis
df = df.assign(x_locatin = x_cg)
df = df.assign(y_location = y_cg)

#print(df_x)
# df_x is a dataset made up of 187 rows and 7 columns corresponding to the attributes

#print(df_y)
# df_y is a dataset made up of 187 rows and 7 columns corresponding to the attributes

#print(df)
# df is a dataset made up of 187 rows and 16 columns ( 7 attributes per axis + one location per axis)



In this next step, the fed datasets will be divided in training and testing datasets.

In [579]:
coef = 0.2

# x_cg and y_cg will be the respective results for the testing and training of the 7 atributes fed Neural Network
# y_2d will be the respective result for the testing and training of the 14 atributes fed Neural Network
y_2d = pd.concat([x_cg, y_cg], axis = 1, )
print(y_2d)

in_x_train, out_x_train, in_x_test, out_x_test = train_test_split(df_x, x_cg, test_size = coef)
in_y_train, out_y_train, in_y_test, out_y_test = train_test_split(df_y, y_cg, test_size = coef)
in_2d_train, out_2d_train, in_2d_test, out_2d_test = train_test_split(df, y_2d, test_size = coef)




     x_location  y_location
0    408.893870  172.336426
1    401.468215  227.810442
2    316.728379  189.808557
3    270.427232  202.912656
4    266.496002  217.327164
..          ...         ...
121  488.392067  196.797410
122  262.564773  156.611508
123  479.656001  203.349459
124  535.566821  271.927573
125  548.670919  308.182245

[187 rows x 2 columns]


Now, the Neural Network model will be created by using the Tensorflow library.

In [580]:
import tensorflow as tf

input_shape = [x_train.shape[1]]

model = tf.keras.Sequential([
tf.keras.layers.Dense(units=2,input_shape=input_shape)])
 
# printing a summary in order to check if it works
model.summary()

NameError: name 'x_train' is not defined